# Modular Empirical Size Analysis System Demo

This notebook demonstrates the usage of the newly refactored, modular Python system for Empirical Size Analysis of Hybrid LogNormal-Pareto distributions.

## Overview of the New System

The original monolithic notebook has been refactored into a clean, maintainable package structure under `src/`:

*   **`src.distribution`**: Handles data generation for the Hybrid LogNormal-Pareto distribution.
*   **`src.estimation`**: Contains robust parameter estimation logic (MLE, Hill, Method of Moments).
*   **`src.statistics`**: Computes the vector of auxiliary statistics used for testing.
*   **`src.testing`**: Implements the hypothesis testing logic (Test 1, Test 1opt) and Beta selection.
*   **`src.simulation`**: Orchestrates the full Monte Carlo simulation using parallel processing.

This modular approach allows for easier testing, maintenance, and reuse of individual components.

## 1. System Initialization and Configuration

First, we need to set up the environment and import our new modules. We ensure the `src` directory is in the Python path.

In [ ]:
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Add the current directory to the path so we can import from src
sys.path.append(os.getcwd())

# Import our new modular components
from src.distribution import HybridLogNormalPareto, HybridParams
from src.estimation import ParameterEstimator
from src.statistics import AuxiliaryStatistics
from src.testing import HypothesisTest
from src.simulation import SimulationRunner

# Configure plotting
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (10, 6)

print("System initialized and modules loaded successfully.")

## 2. Data Generation (Distribution Module)

The `src.distribution` module encapsulates the logic for generating the hybrid data. It ensures that the threshold $\tau$ is placed exactly at the specified percentile.

We define our parameters using the `HybridParams` dataclass, which provides type safety and clarity.

In [ ]:
# Define parameters
params = HybridParams(
    mu=9.0,
    sigma=np.sqrt(1.17),  # Note: Input is standard deviation
    alpha=3.05,
    tau=32000,
    tau_perc=0.82
)

# Generate a sample
n_samples = 5000
data = HybridLogNormalPareto.generate_sample(n_samples, params, seed=42)

print(f"Generated {len(data)} samples.")
print(f"Min: {data.min():.2f}, Max: {data.max():.2f}")

# Verify percentile placement
actual_percentile = (np.sum(data <= params.tau) / n_samples)
print(f"Target Percentile: {params.tau_perc:.4f}")
print(f"Actual Percentile: {actual_percentile:.4f}")

# Visualize
plt.figure(figsize=(12, 6))
sns.histplot(data, bins=100, log_scale=(True, False))
plt.axvline(params.tau, color='red', linestyle='--', label=f'Tau = {params.tau}')
plt.title("Generated Hybrid LogNormal-Pareto Distribution (Log Scale X)")
plt.legend()
plt.show()

## 3. Parameter Estimation (Estimation Module)

The `src.estimation` module provides tools to estimate the parameters ($\mu, \sigma, \alpha$) back from the data, given a threshold $\tau$. It uses robust methods for the Pareto tail.

In [ ]:
# Estimate parameters assuming we know the true tau
mu_hat, sigma2_hat, alpha_hat = ParameterEstimator.estimate_parameters(params.tau, data)

print("Parameter Estimation Results:")
print(f"{'Parameter':<10} {'True':<10} {'Estimated':<10} {'Error %':<10}")
print("-" * 45)
print(f"{'Mu':<10} {params.mu:<10.4f} {mu_hat:<10.4f} {abs(mu_hat-params.mu)/params.mu*100:<10.2f}")
print(f"{'Sigma^2':<10} {params.sigma**2:<10.4f} {sigma2_hat:<10.4f} {abs(sigma2_hat-params.sigma**2)/params.sigma**2*100:<10.2f}")
print(f"{'Alpha':<10} {params.alpha:<10.4f} {alpha_hat:<10.4f} {abs(alpha_hat-params.alpha)/params.alpha*100:<10.2f}")

## 4. Auxiliary Statistics (Statistics Module)

The `src.statistics` module computes a comprehensive vector of statistics (percentiles, ratios, inequality measures) that capture the shape of the distribution. These are used as the basis for the hypothesis test.

In [ ]:
# Compute statistics for a specific moment order (e.g., 4)
order = 4
stats_vector = AuxiliaryStatistics.compute_vector(data, order)

print(f"Auxiliary Statistics Vector (Order {order}):")
print(f"Shape: {stats_vector.shape}")
print(f"First 5 values: {stats_vector[:5]}")

# The vector includes percentiles, ratios, Gini, Theil, etc.
# We can inspect specific components if needed by using the compute method directly
single_stat = AuxiliaryStatistics.compute(data, 1) # First statistic
print(f"First statistic (25th percentile): {single_stat}")

## 5. Hypothesis Testing (Testing Module)

The `src.testing` module contains the core logic for:
1.  **Evaluating Tau**: Finding the threshold that minimizes the distance between observed and simulated statistics.
2.  **Beta Selection**: Using cross-validation to find the optimal regularization parameter for the weighted test.
3.  **Test Statistics**: Computing the unweighted (Test 1) and weighted (Test 1opt) statistics.

In [ ]:
# 1. Evaluate Optimal Tau
# We search over a grid of percentiles
tau_grid_percentiles = np.arange(70, 90, 2)
tau_values = np.percentile(data, tau_grid_percentiles)

print("Evaluating optimal tau...")
best_tau = HypothesisTest.evaluate_tau(data, tau_values, order=4, S=50) # Reduced S for demo speed
print(f"Best Tau: {best_tau:.2f}")
print(f"True Tau: {params.tau:.2f}")

# 2. Beta Selection (Cross-Validation)
# This is computationally intensive, so we run it on a small scale here
c_values = np.array([10, 1.0, 0.1])
N = len(data)

print("\nSelecting optimal beta...")
best_beta = HypothesisTest.select_optimal_beta(
    N=N,
    c_values=c_values,
    moment_order=4,
    S=20,  # Reduced for demo
    R=20,  # Reduced for demo
    true_data=data,
    best_tau=best_tau,
    tau_grid=tau_grid_percentiles,
    seed=42
)
print(f"Selected Beta: {best_beta}")

## 6. Full Simulation (Simulation Module)

The `src.simulation` module puts it all together. It runs the full Monte Carlo experiment to calculate the empirical size of the tests. It uses `joblib` for parallel execution.

This replaces the `application_analysisl` and `application_analysisl_parallel` functions from the original notebook.

In [ ]:
# Configure the simulation runner
# We use small parameters for this demo to ensure it runs quickly
runner = SimulationRunner(
    n_data=1000,       # Smaller sample size
    M=5,               # Only 5 Monte Carlo repetitions
    S=50,              # Fewer simulated stats
    R=50,              # Fewer bootstrap reps
    orders=[4],        # Only test order 4
    n_jobs=-1,         # Use all available cores
    verbose=True
)

print("Starting simulation...")
results_df = runner.run()

print("\nSimulation Results:")
display(results_df)

# Visualize the results (if we had more orders/rows)
if len(results_df) > 0:
    plt.figure(figsize=(8, 5))
    results_melted = results_df.melt(
        id_vars=['Order'], 
        value_vars=['Test 1', 'Test 1opt'],
        var_name='Test Type',
        value_name='Empirical Size'
    )
    sns.barplot(data=results_melted, x='Order', y='Empirical Size', hue='Test Type')
    plt.axhline(0.05, color='red', linestyle='--', label='Nominal Size (0.05)')
    plt.title("Empirical Size by Test Type")
    plt.legend()
    plt.show()

## 7. Comparison with Original Implementation

### Key Improvements

1.  **Modularity**: The logic is no longer trapped in a single 1700-line notebook cell. Each component (Distribution, Estimation, Statistics, Testing) is in its own file.
2.  **Parallelization**: We replaced the manual `multiprocessing` code with `joblib`, which is more robust and standard in the Python data science ecosystem.
3.  **Type Safety**: The new code uses Python type hints (`typing`), making it easier to understand what data is being passed around.
4.  **Maintainability**: The `HybridParams` dataclass prevents "magic numbers" and argument confusion.

### Mapping

| Original Notebook Function | New System Component |
| :--- | :--- |
| `generate_hybrid_sample` | `src.distribution.HybridLogNormalPareto.generate_sample` |
| `estimate_theta_improved` | `src.estimation.ParameterEstimator.estimate_parameters` |
| `compute_auxiliary_statistics` | `src.statistics.AuxiliaryStatistics.compute_vector` |
| `select_optimal_beta` | `src.testing.HypothesisTest.select_optimal_beta` |
| `application_analysisl` | `src.simulation.SimulationRunner.run` |
| `application_analysisl_parallel` | `src.simulation.SimulationRunner.run` (via `n_jobs` param) |